In [ ]:
pip install GitPython

In [ ]:
import datetime

In [ ]:
def printStatus(index, size):
    print("{0}% Completed samples".format((index / size) * 100))

In [ ]:
from git import Repo
import pandas as pd
import numpy as np
import os
import fnmatch
from statistics import mean
import shutil
import git
import math

### To run this code you need to download the 'rsm.jar' found in 'https://dibt.unimol.it/report/readability/', and it should be stored at root path

In [ ]:
def getReadability(file):
    file = file.replace("/", "\\")
    r = os.popen(f"java -jar rsm.jar {file}").read().replace("\n", "")   
    try:
        readability = float(r.split("\t")[-1])
        if math.isnan(readability):
            return 0
        else:
            return readability
    except:
        print(r)
        return 0

In [ ]:
samples = pd.read_csv("../1-GettingQuestions/sampleQuestions.csv")

In [ ]:
# samples = samples[samples["framework"] == "SAP-samples"]

In [ ]:
samples["full_name"] = samples["framework"] + "/" + samples["path"]

In [ ]:
samples = samples["full_name"].unique().tolist()

In [ ]:
# samples = samples[samples.index("googlesamples/android-testdpc")+1:]

In [ ]:
len(samples)

In [ ]:
for sample in samples:
    print(sample)
    
    file_address = f"metrics/{sample}.csv"
    metrics = pd.read_csv(file_address)
    metrics["Scalabrino Readability"] = 0

    last_commit_result = ("0",0)
    printStatus(samples.index(sample), len(samples))
    
    arquivos = dict()
    
    for index in range(0, len(metrics)):
        print(f"{index}/{len(metrics)} of line analized from {sample}")
        print("inicio análise da linha " + str(index))
        inicio = datetime.datetime.now()


        current_commit = metrics.iloc[index]["commitSha"]
        
#         print("Checking if is the same commit")
        if current_commit == last_commit_result[0]:
            metrics.iloc[index, len(metrics.columns.values) - 1] = last_commit_result[1]
            metrics.to_csv(file_address, index=False)
#             print("Fim análise porque é o mesmo commit")
            continue
        
        repository = Repo("repositories/"+sample)

        java_files = list()
        for file in repository.commit(current_commit).stats.files:
            if "java" in file.split("."):
                java_files.append(file)


        if not len(java_files):
            metrics.iloc[index, len(metrics.columns.values) - 1] = last_commit_result[1]
            metrics.to_csv(file_address, index=False)
            continue

        else:
            repository.git.checkout(current_commit, "-f")
            print(f"{len(java_files)} files to be analyzed")
            for file in java_files:
                if("java" in file.split(".")):
                    readability = getReadability(f"repositories/{sample}/{file}")
                    arquivos[file] = readability
                    
            last_commit_result = (current_commit, mean(list(arquivos.values())) )

            metrics.iloc[index, len(metrics.columns.values) - 1] = last_commit_result[1]
            metrics.to_csv(file_address, index=False)
            print("Fim análise com a execução da readability")
            fim = datetime.datetime.now()
            print("Tempo gasto: " + str((fim - inicio).total_seconds()))